In [ ]:
#dependencies

import pandas as pd
from IPython.display import Markdown, display
from tabulate import tabulate

# Summary

We are developing a classification model using a KNN machine learning algorithm to categorize Airbnb listing prices into various ranges. The goal is to accurately assign each listing to a price bracket, starting at $0 and increasing in $50 increments up to $350 and above.

Our research question for this analysis is: *How do geographical distribution, review trends, and the introduction of Local Law 18 influence the pricing and popularity of Airbnb listings across different boroughs in New York City?*

The classifier is designed to help potential guests estimate the price range of a rental before detailed information is provided. The performance of our model on the test data set indicates varying degrees of precision and recall across different price categories, with an overall accuracy of 0.51. Given that the data set consists of 7926 instances, the model shows promise, but there is significant room for improvement. Particular attention should be given to enhancing the performance for categories that currently show lower F1-scores. Further analysis is needed to understand the features that most affect price prediction accuracy and to refine the model for better performance across all price ranges.

# Introduction

Airbnb has been one of the most popular short-term stays and rental online services since 2008, expanding to over a thousand cities in less than a year [@Geron_2009]. Owing to its exponential growth however, many metropolises such as Paris, London, Berlin, and New York City started introducing special rules and laws for creating a listing on Airbnb due to its impact on housing shortages and rental markets [@Tun_2023]. In September 2023, New York enforced a measure called Local Law 18 [@Oladipo_2023], that was essentially a de facto ban on short-term rentals that would force the amount of listings to decrease 85% within the month [@Chan_2023], and decrease over 90% as of 2024 from what they were in 2022 [@Bellafante_2024]. Consequently, this led average prices to surge, and led us to want to investigate factors of New York Airbnbs and how they impact listing prices.

We were able to obtain a workable and descriptive dataset about New York Airbnbs from [insideairbnb.com](http://insideairbnb.com/new-york-city/) [@New_York_City] that gave insight into every listing from 2011 to 2023 and their reviews. The fields present in our dataset are:

* ID: Unique identifier for listing
* Name: Name of listing, followed by rating and type (e.g. 1 bed 1 bath)
* Host ID: Unique identifier for host
* Host Name: Name of host
* Neighbourhood: Neighbourhood where listing is located
* Neighbourhood Group: Borough where listing is located
* Latitude: Latitudinal coordinate of listing location
* Longitude: Longitudinal coordinate of listing location
* Room Type: Listing space type (Private room, Entire home / Apt., etc.)
* Price: Price per Night in USD
* Minimum Nights: Minimum nights required to stay at the rental
* Number of Reviews: Total number of reviews on the listing
* Last Review: Date of latest review on the rental
* Reviews Per Month: Avg. monthly number of reviews on the listing
* Calculated Host Listings Count: Number of listings the corresponding host has
* Availability 365: Number of days in a year listing is available for booking

Here we intend to use a machine learning algorithm to answer the research question:

**How do geographical distribution, review trends, and the introduction of Local Law 18 influence the pricing and popularity of Airbnb listings across different boroughs in New York City?**

There have been multiple studies looking into how the Airbnb ban impacts NYC rental [@Fields_2023] and hotel prices [@Kelly_2023], but not as much into the Airbnb prices themselves. Since our investigation will be solely based on data up until 2023, it can be used glean exactly how these restrictions impacted Airbnb prices in NYC when compared with information from 2024 onwards (i.e. was it because the number of listings per host went down? Or was it because the minimum number of nights required to stay went up? and more questions like that). As Airbnb prices can range to various amounts, we will look to *bin prices into different categories in increments of 50*. This will help us provide a more accurate range of price recommendations when looking at different Airbnb listings.

# Methods

The Python programming language [@Python] and the following Python packages were primarily used to perform the analysis:
pandas [@pandas], scikitlearn [@scikit-learn], click [@click], as well as Quarto [@Allaire_Quarto_2022].

As previously mentioned, we loaded data from [insideairbnb.com](http://insideairbnb.com/new-york-city/) [@New_York_City].

## Takeaways From Preliminary EDA

- We have 39627 rows and 18 features which includes `price`. We decided it was a good idea to generate a `price_category` variable as our target variable that classified the price into bins / brackets starting at $0 and increasing in $50 increments up to $350 +.

- We identified that `name` is text type data which could provide some valuable insights. We can also infer that any id information (`id` and `host_id`) and variables such as `host_name` will not provide any key information, thus, we can drop them for our data analysis.

- The column `host_name` had *15* null values but it is not a roadblock since we dropped this column. Our `reviews_per_month` column and `last_review` column have *11480* null values each. This suggests that when a airbnb listing doesn't have a review, it was listed as a null value in the data. To fix this we could impute 0 into the nulls for `reviews_per_month`. For `last_review`, we decided it would be appropriate to drop the column as some of the information it provides is stored in reviews per month.

- `license` has a significant number of null values with 35268. It might provide some interesting insights into price as a license is proof that the airbnb owners comply with the regulatory standards, but since the nature of these missing values is unknown, we cannot be sure if the null values means the listing has no license. Due to this imbalance in the data, we decided it best to drop this column. However, it could be worth exploring the relationship between license and other variables in future analyses.

## Next Steps for Feature Engineering

In order to prepare our data for further analysis we performed some preliminary feature engineering:
- Converted `id` and `host_id` into object datatypes to prepare them to be dropped.
- Imputed zeros into the `reviews_per_month` null values.
- Segregated fields into target column, categorical, numerical, text data, and drop data.
- Created a price category variable.

After this cleaning, we split the data into training and test sets.

## Spread of `price` Visualized

We plotted a histogram for our `price` variable to graph its' spread on @fig-price-plot below.

![Price Histogram (Outliers Removed).](../results/figures/price_histogram.jpg){#fig-price-plot width=80%}

## Relationships between Predictors Visualized

We examined the correlations between the numeric data we had present to help us understand which predictors could be used in tandem. The top 10 crorrelations are displayed in @tbl-corr-ranked below, and we plotted a correlation heat map on @fig-corr-plot to visualize it:


In [ ]:
#| label: tbl-corr-ranked
#| tbl-cap: Top 10 Correlations Ranked.

corr_ranked = pd.read_csv("../results/tables/correlations_ranked.csv")
Markdown(corr_ranked.to_markdown(index = False))

![Correlation Heat Map of Numerical Predictors of Airbnb Price.](../results/figures/corr_heat_map.jpg){#fig-corr-plot width=80%}

We see from @fig-corr-plot that `number_of_reviews`, `reviews_per_month`, and `number_of_reviews_ltm` appear to have the strongest correlations between them, indicating they interact with each other more.

We also visualized the relationships between some of our other predictors to view the spread of our data and the impact of every predictor on `price` (to extend it to the impact it has on `price category`).

We plotted the physical location by latitude and longitude of every listing on @fig-location-plot, placed alongside an outline of a map of NYC to see the concentration of listings between boroughs.

![Map with Distribution of Listings by Location and Price.](../results/figures/listing_locations.jpg){#fig-location-plot}

We also created scatterplots of `price` against the `number_of_reviews` as well as the `reviews_per_month` on @fig-price-v-reviews-plot and @fig-price-v-reviews-per-month-plot respectively to understand how the distribution compares for the two - considering both predictors were largely correlated with each other.

![Price vs Number of Reviews Coloured by Room Type Scatterplot.](../results/figures/price_vs_reviews.jpg){#fig-price-v-reviews-plot}

![Price vs Reviews Per Month Coloured by Room Type Scatterplot.](../results/figures/price_vs_reviews_per_month.jpg){#fig-price-v-reviews-per-month-plot}

We then created side-by-side boxplots to compare `price` (as well as log price for clarity) against the `neighbourhood_group` or the NYC borough, as well as `room_type` to view the isolated relationships between price and those 2 predictors and the spread of the listings in each category. They can be see in @fig-neighbourhood-plot and @fig-room-type-plot below:

![Log Price and Price by Neighbourhood Group Boxplot.](../results/figures/neighbourhood_groups_boxplots.jpg){#fig-neighbourhood-plot}

![Log Price and Price by Room Type Boxplot.](../results/figures/room_type_boxplots.jpg){#fig-room-type-plot}

After this, we moved onto creating our preprocessor and model.

## Defining Transformations and Model

We defined transformations on our data, using `StandardScaler` on our numerical data, `OneHotEncoder` on our categorical data, `CountVectorizer` on our text data. We also implement a Dummy Regressor model as a baseline to assess our model with.

We then proceed to fit a K-Nearest-Neighbours Classification model using `KNeighborsClassifier` from [@scikit-learn].

## Re-evaluating Model with Hyperparameter Optimization

We decided to also use `RandomizedSearchCV` to choose hyperparameters that would yield an optimal model for our data.

# Results

We generated classification reports for all 3 models we fit - the Dummy regressor, the original KNN and the KNN after hyperparameter optimization to compare their performance and accuracy.

## Classification Report for Dummy Model


In [ ]:
#| label: tbl-dummy-clf
#| tbl-cap: Classification Report for Dummy Regressor.

dummy_clf = pd.read_csv("../results/tables/dummy_classification_report.csv")
dummy_clf.rename(columns={"Unnamed: 0": "Criterion"}, inplace=True)
dummy_accuracy = dummy_clf.iloc[8, 1]
Markdown(dummy_clf.to_markdown(index = False))

We see in @tbl-dummy-clf here that the accuracy for the dummy model is `{python} round(dummy_accuracy, 2)`.

## Classification Report for KNN Model


In [ ]:
#| label: tbl-knn-clf
#| tbl-cap: Classification Report for KNN Classification Model.

knn_clf = pd.read_csv("../results/tables/knn_classification_report.csv")
knn_clf.rename(columns={"Unnamed: 0": "Criterion"}, inplace=True)
knn_accuracy = knn_clf.iloc[8, 1]
Markdown(knn_clf.to_markdown(index = False))

We see in @tbl-knn-clf here that the accuracy for the KNN model is `{python} round(knn_accuracy, 2)`, which is better than our dummy model, but we still want to see if we can do better with another choice of neighbours. Upon hyperparameter optimization, we found that 15 neighbours proved to be the optimal choice.

## Classification Report After Hyperparameter Optimization


In [ ]:
#| label: tbl-hyperparam-clf
#| tbl-cap: Classification Report for KNN after Hyperparameter Optimization.

hyperparam_clf = pd.read_csv("../results/tables/hyperparam_classification_report.csv")
hyperparam_clf.rename(columns={"Unnamed: 0": "Criterion"}, inplace=True)
hyperparam_accuracy = hyperparam_clf.iloc[8, 1]
Markdown(hyperparam_clf.to_markdown(index = False))

From @tbl-hyperparam-clf we see that this optimization processes improved the accuracy of our model from around `{python} round(knn_accuracy, 2)` to `{python} round(hyperparam_accuracy, 2)`.

# Discussion

Our investigation utilized an KNN machine learning algorithm to estimate pricing categories for Airbnb listings in New York City. The dataset was divided using an 80/20 train-test split, and we processed features as either numerical or categorical to forecast the prices within predetermined ranges. The model achieved an accuracy (F1-Score) of about `{python} round(hyperparam_accuracy, 2)` as indicated in the classification report above, which is not the best however, considering the complexity of the pricing structure in the short-term rental market it performed decently. 

The level of accuracy attained was anticipated to some extent. We undertook a meticulous selection of predictive variables and addressed missing values to enhance the model's precision. However, the modest F1-score (a good F1 score is 0.7 or higher) indicates the model isn't the most accurate fit and could benefit from improvements such as better feature selection and feature engineering. This could be done through performing feature importance analysis and working with that information to further develop our model. We created a model with a KNN to test.

At present, our feature importance metrics may be distorted due to the numerous variables generated through one-hot encoding. For subsequent analyses, we might consider exploring alternative methods to evaluate feature significance, such as aggregating the importance of dummy variables back to the original categorical variables, or employing encoding techniques that preserve more information about the categorical variable's cardinality and inherent ordering. This approach could potentially provide a more accurate reflection of each feature's true contribution to the predictive model.

The implications of such a model are significant. For renters, it could mean a more informed decision-making process, as they could benchmark individual rental prices against the broader market. Landlords might utilize the model to set competitive prices, ensuring their offerings are in line with current market conditions.

Moving forward, this study opens up several avenues for further inquiry. One such area is the incorporation of temporal dynamics into the model, as pricing could be influenced by seasonal trends or significant events. Continuous refinement of the model is essential to maintain its relevance, as the short-term rental market is subject to frequent fluctuations. Additionally, investigating the outliers and misclassified instances could provide insights into the limitations of the current model and guide enhancements in predictive accuracy.

# References